In [ ]:
import Data_Collection as data
import numpy as np
import matplotlib.pyplot as plt

def time_series(data_dict, plot_title:str='Pressure Response'):
    plt.figure(figsize=(15, 10))
    plt.rc('xtick', labelsize=20) 
    plt.rc('ytick', labelsize=20) 

    for key in data_dict['data'].keys():
        p_arr = data_dict['data'][key]['pressures']
        time_arr = data_dict['data'][key]['times']

        plt.plot(time_arr, (np.array(p_arr) - np.mean(p_arr)))

    plt.legend(data_dict['specifications']['stations'],
        bbox_to_anchor=(1.125, 1.0),
        loc='upper right', fontsize=20)

    filter_num = str(data_dict['specifications']['filter'])
    plt.title(plot_title + ' %s' % filter_num, fontsize=40)

    time_unit = str(data_dict['specifications']['units']['times'])
    plt.xlabel('Time (%s)' % time_unit, fontsize=40)

    pressure_unit = str(data_dict['specifications']['units']['pressures'])
    plt.ylabel('Pressure (%s)' % pressure_unit, fontsize=40)

In [ ]:
path = '/Users/namsonnguyen/OneDrive/Pressures/filter5/Field_Experiments/'
file = 'jul25_field_boom.pkl'
new_dict = data.reformat(path, file)

#time_series(new_dict)

In [ ]:
delta_data = {}
delta_data['specifications'] = new_dict['specifications']
stations = new_dict['specifications']['stations']
for i in range(len(stations)):
    for j in range(i+1, len(stations)):
        key = str(stations[i] + '-' + stations[j])
        new_dict['data'][key] = 'array of values'
        print(key)



In [ ]:
def excess_path_length(filter_number, p_data):
    filter_number = b'IA=%d' % filter_number
    delta_data = {}

    for i in range(len(p_data)):
        for j in range(i+1, len(p_data)):

            dataset_1 = np.array(p_data[i][filter_number]['pressures']) - np.mean(p_data[i][filter_number]['pressures'])
            dataset_2 = np.array(p_data[j][filter_number]['pressures']) - np.mean(p_data[j][filter_number]['pressures'])

            delta_12_name = str((sea_mammals[i])[0:3]) + '-' + str((sea_mammals[j])[0:3])   # Serves as the delta_12 key for delta_data dictionary
            delta_data[delta_12_name] = (dataset_1 - dataset_2) * (L_norm / p_norm)

    return delta_data

def excess_path_length(data_collection, L_norm:float=2000, p_norm:float=1,
                       L_norm_units:str='mm', p_norm_units:str='bar'):
    delta_data = {}
    delta_data['specifications'] = data_collection['specifications']

    stations = new_dict['specifications']['stations']
    for i in range(len(stations)):
        for j in range(i+1, len(stations)):
            
            p_arr1 = np.array(data_collection['data'][stations[i]]['pressures']) - np.mean(data_collection['data'][stations[i]]['pressures'])
            p_arr2 = np.array(data_collection['data'][stations[j]]['pressures']) - np.mean(data_collection['data'][stations[j]]['pressures'])

            key = str(stations[i] + '-' + stations[j])
            delta_data['data'][key] = (p_arr1 - p_arr2) * (L_norm / p_norm)

    return delta_data